# Neural networks

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
#from keras import utils

#from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

# for modeling
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

TypeError: validate_params() got an unexpected keyword argument 'prefer_skip_nested_validation'

In [4]:
# Load density data

MMS2_n = pd.read_csv('MMS2_n.csv',
                     header = 0,names = ['times','n','date'],
                     parse_dates=['date'])


# load temperature data
MMS2_Tperp = pd.read_csv('MMS2_Tperp.csv',
                     header = 0,names = ['times','Tperp','date'],
                     parse_dates=['date'])

MMS2_Tpara = pd.read_csv('MMS2_Tpara.csv',
                     header = 0,names = ['times','Tpara','date'],
                     parse_dates=['date'])


In [ ]:
# Magnteosphere time periods were identified by eye. 
# Find matching indices to classify density data set into being either 1 magnetosphere, or 0 not magnetosphere


# Identified magnetosphere time periods





NB_st_time = '2020-06-17/20:30:00'

NB_end_time = '2020-06-17/23:00:53'

NB_st_idx = MMS2_n.date.searchsorted(NB_st_time)
NB_end_idx = MMS2_n.date.searchsorted(NB_end_time)

# def MSP_classification(density_arr, )








print(NB_st_idx)

print(NB_end_idx)




# create an array the length of the classified data, fill with 0's
MSP = np.zeros(len(MMS2_n.date))






Magnetosphere_st_stop_times = ['2020-06-17/20:30:00','2020-06-17/21:16:31',
'2020-06-17/21:32:41','2020-06-17/21:35:30',
'2020-06-17/21:38:54','2020-06-17/21:41:40', 
'2020-06-17/21:49:00','2020-06-17/21:53:39',
'2020-06-17/21:56:49','2020-06-17/22:00:36',
'2020-06-17/22:04:52','2020-06-17/22:10:44',
'2020-06-17/22:13:28', '2020-06-17/22:16:47',
'2020-06-17/22:20:06','2020-06-17/22:23:55',
'2020-06-17/22:27:45','2020-06-17/22:30:56',
'2020-06-17/22:34:34','2020-06-17/22:37:37',
'2020-06-17/22:39:17','2020-06-17/22:43:45',
'2020-06-17/22:46:41','2020-06-17/22:49:33',
'2020-06-17/22:55:59','2020-06-17/23:00:53']


# too lazy to reformat the original list, so specify that start times are every other 
# element starting at 0th index, end times are every other index starting at 1st index

st_times = Magnetosphere_st_stop_times[::2]
end_times = Magnetosphere_st_stop_times[1::2]

for t in range(len(st_times)):
    st_idx_1 = MMS2_n.date.searchsorted(st_times[t])
    #print('start index:',st_idx_1)
    end_idx_1 = MMS2_n.date.searchsorted(end_times[t])
    #print('end index:',end_idx_1)
    MSP[st_idx_1:end_idx_1] = 1
    #print('\n')
    

    

# save to a .csv so I can visually inspect the result
#pd.DataFrame(MSP).to_csv('MSP_classifier.csv', index=False) 

#NB_n_DF = MMS2_n.iloc[NB_st_idx:NB_end_idx]


n = np.array(MMS2_n.n[NB_st_idx:NB_end_idx])


#print(n)

# NB_n_DF['MSP_classifier'] = MSP[NB_st_idx:NB_end_idx]

NN_DF = pd.DataFrame(data = {"n": MMS2_n.n[NB_st_idx:NB_end_idx],
                            "MSP":MSP[NB_st_idx:NB_end_idx]})


NN_DF


NN_DF['MSP'].value_counts()

print(36800/(36800+23550))


In [ ]:
# splitting data

X = NB_DF.drop(['MSP'], axis=1)

y = NB_DF['MSP']



X = np.array(X)
y = np.array(y)

In [ ]:
# build a model
model = Sequential()
model.add(Dense(16, input_shape=(X.shape[1],), activation='relu')) # Add an input shape! (features,)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary() 

# compile the model
model.compile(optimizer='Adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 10 consecutive epochs.  
es = EarlyStopping(monitor='val_accuracy', 
                                   mode='max', # don't minimize the accuracy!
                                   patience=10,
                                   restore_best_weights=True)

# now we just update our model fit call
history = model.fit(X,
                    y,
                    callbacks=[es],
                    epochs=80, # you can set this to a big number!
                    batch_size=10,
                    validation_split=0.2,
                    shuffle=True,
                    verbose=1)

In [ ]:
history_dict = history.history
# Learning curve(Loss)
# let's see the training and validation loss by epoch

# loss
loss_values = history_dict['loss'] # you can change this
val_loss_values = history_dict['val_loss'] # you can also change this

# range of X (no. of epochs)
epochs = range(1, len(loss_values) + 1) 

# plot
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()